# Loading (made by Dengsong Wang)

In [1]:
import psycopg2
import pandas as pd
import os

In [2]:
# Reading in transformed tables
fact_table = pd.read_csv("../transform/fact_table.csv")

date_dimension = pd.read_csv("../transform/date_dimension.csv")
location_dimension = pd.read_csv("../transform/location_dimension.csv")
real_estate_dimension = pd.read_csv("../transform/real_estate.csv")
regional_weather_summaries = pd.read_csv("../transform/regional_weather_summaries.csv")
weather_station_summary = pd.read_csv("../transform/weather_station_summary.csv")

In [3]:
regional_weather_summaries

,region_summ_key,region_mean_temp,region_max_temp,region_min_temp,region_mean_precip,region_mean_snow,mean_snow_grnd_last_day,location_key,date_key
0,0,-7.383402,11.118672,-34.675104,10.042917,10.700000,7.191781,9,0
1,1,-6.802490,7.707025,-25.254357,11.007500,17.420000,11.261364,9,1
2,2,-1.088477,14.159259,-19.236214,19.475732,26.260656,4.768116,9,2
3,3,4.472016,22.780408,-9.363821,36.970248,15.016923,0.388889,9,3
4,4,9.636364,26.234979,-3.077273,48.488136,2.229091,0.044444,9,4
...,...,...,...,...,...,...,...,...,...
1737,1737,-1.275000,15.710714,-18.221429,54.061538,23.450000,8.636364,11,129
1738,1738,-13.167857,2.596429,-35.492857,12.666667,9.950000,19.153846,11,130
1739,1739,-22.457143,-0.689286,-44.425000,19.766667,21.100000,31.153846,11,131
1740,1740,-14.571429,-0.164286,-28.867857,18.116667,12.950000,39.500000,11,132


In [6]:
# change dbname, user, password, host, port to your own if you want to run it on your own database
def create_conn():
    connection = psycopg2.connect(
        dbname="4142_Database",
        user="postgres",
        # password = os.getenv("DB_PASS"),
        password="Oatmeal98",
        host="localhost",
        port="5432"
    )
    return connection

conn = create_conn()
cursor = conn.cursor()

# create a table 
def create_tables(connection, query_file_path="init_conn.session.sql"):
    cursor = connection.cursor()
    
    with open(query_file_path, 'r') as file_o:
        query = file_o.read()
    
    cursor.execute(query)
    connection.commit()
    cursor.close()
    
create_tables(conn)
conn.close()

In [8]:
# load the df into the table
from sqlalchemy import create_engine

def get_engine(connection):
    return create_engine(f"postgresql+psycopg2://{connection.info.user}:{connection.info.password}@{connection.info.host}:{connection.info.port}/{connection.info.dbname}")

def load_dataframe(dataframe, table_name, connection):
    engine = get_engine(connection)
    dataframe.to_sql(table_name, engine, if_exists='replace', index=False, schema='public', method='multi')

conn = create_conn()
load_dataframe(location_dimension, 'location_dimension', conn)
load_dataframe(date_dimension, 'date_dimension', conn)
load_dataframe(real_estate_dimension, 'real_estate_dimension', conn)
load_dataframe(regional_weather_summaries, 'regional_weather_summary', conn)
load_dataframe(fact_table, 'facttable', conn)

conn.close()

In [6]:
# validate the df is loaded successfully
def show_table(table,connection):
    # eng = get_engine(connection)
    # curs = connection.cursor()
    query = f"SELECT * FROM \"{table}\""
    dataframe = pd.read_sql_query(query, connection)
    connection.close()
    return dataframe

conn = create_conn()
table_df = show_table('facttable',conn)
conn.close()

C:\Users\jonat\AppData\Local\Temp\ipykernel_25036\1883327639.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql_query(query, connection)
